In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

hisseler=["ECILC"]  #Hisse kodlarını istediğiniz sayıda yazabilirsiniz

for i in hisseler:
    hisse=i
    tarihler=[]
    yıllar=[]
    donemler=[]
    grup=[]

    url1="https://www.isyatirim.com.tr/tr-tr/analiz/hisse/Sayfalar/sirket-karti.aspx?hisse="+hisse
    r1=requests.get(url1)
    soup=BeautifulSoup(r1.text,"html.parser")
    secim=soup.find("select",id="ddlMaliTabloFirst")
    secim2=soup.find("select",id="ddlMaliTabloGroup")
    
    try:
        cocuklar=secim.findChildren("option")
        grup=secim2.find("option")["value"]
        
        for i in cocuklar:
            tarihler.append(i.string.rsplit("/"))
        for j in tarihler:
            yıllar.append(j[0])
            donemler.append(j[1])

        if len(tarihler)>=4:
            parametreler=(
                ("companyCode",hisse),
                ("exchange","TRY"), #Eğer dolar bazlı istiyorsanız "TRY" yerine "USD" yazınız
                ("financialGroup",grup),
                ("year1",yıllar[0]),
                ("period1",donemler[0]),
                ("year2",yıllar[1]),
                ("period2",donemler[1]),
                ("year3",yıllar[2]),
                ("period3",donemler[2]),
                ("year4",yıllar[3]),
                ("period4",donemler[3]))
            url2="https://www.isyatirim.com.tr/_layouts/15/IsYatirim.Website/Common/Data.aspx/MaliTablo"
            r2=requests.get(url2,params=parametreler).json()["value"]
            veri=pd.DataFrame.from_dict(r2)
            veri.drop(columns=["itemCode","itemDescEng"],inplace=True)           
        else:
            continue
    except AttributeError:
        continue
    
    del tarihler[0:4]
    tumveri=[veri]
        
    for _ in range(0,int(len(tarihler)+1)):
        if len(tarihler)==len(yıllar):
            del tarihler[0:4]
        else:
            yıllar=[]
            donemler=[]
            for j in tarihler:
                yıllar.append(j[0])
                donemler.append(j[1])
            
            if len(tarihler)>=4:
                parametreler2=(
                ("companyCode",hisse),
                ("exchange","TRY"), #Eğer dolar bazlı istiyorsanız "TRY" yerine "USD" yazınız
                ("financialGroup",grup),
                ("year1",yıllar[0]),
                ("period1",donemler[0]),
                ("year2",yıllar[1]),
                ("period2",donemler[1]),
                ("year3",yıllar[2]),
                ("period3",donemler[2]),
                ("year4",yıllar[3]),
                ("period4",donemler[3]))
                r3=requests.get(url2,params=parametreler2).json()["value"]
                veri2=pd.DataFrame.from_dict(r3)
                try:
                    veri2.drop(columns=["itemCode","itemDescTr","itemDescEng"],inplace=True)
                    tumveri.append(veri2)
                except KeyError:
                    continue
    veri3=pd.concat(tumveri,axis=1)
    baslık=["Bilanço"]
    for i in cocuklar:
        baslık.append(i.string)

    baslıkfark=len(baslık)-len(veri3.columns)

    if baslıkfark!=0:
        del baslık[-baslıkfark:]

    veri3=veri3.set_axis(baslık,axis=1)
    veri3[baslık[1:]]=veri3[baslık[1:]].astype(float)
    veri3=veri3.fillna(0)
    dizin="C:/Users/okand/OneDrive/Desktop/Borsa" #Dosyayı kaydetmek istediğiniz dizini yazın
    veri3.to_excel(dizin+"/{}.xlsx".format(hisse),index=False)

In [3]:
import pandas as pd
import yfinance as yf

veri=yf.download("ECILC.IS",start="2021-04-30", interval="3mo", auto_adjust=True)["Close"] # 2021-04-30
#veri_3ay = veri["Close"].resample("3M").last()
print(veri)


[*********************100%***********************]  1 of 1 completed

Ticker       ECILC.IS
Date                 
2021-04-01   5.125506
2021-07-01   5.282023
2021-10-01   6.774374
2022-01-01   8.619466
2022-04-01   9.044560
2022-07-01  13.945801
2022-10-01  30.055769
2023-01-01  25.634155
2023-04-01  35.447525
2023-07-01  52.292019
2023-10-01  41.493813
2024-01-01  47.194965
2024-04-01  48.469227
2024-07-01  46.156410
2024-10-01  45.924854
2025-01-01  42.335770
2025-04-01  42.142807
2025-07-01  83.599998
2025-10-01  91.199997


In [29]:
from yahooquery import Ticker

# TAVHL.IS için veri çekme
ticker = Ticker("CCOLA.IS")
data = ticker.history(start="2021-04-30", interval="3mo")["adjclose"]

print(data)

symbol    date                     
CCOLA.IS  2021-04-01                    75.687485
          2021-07-01                    82.833595
          2021-10-01                    86.394753
          2022-01-01                   114.347435
          2022-04-01                   126.055351
          2022-07-01                   131.548111
          2022-10-01                   199.765457
          2023-01-01                   199.081329
          2023-04-01                   256.939056
          2023-07-01                   386.156555
          2023-10-01                   514.125305
          2024-01-01                   565.488953
          2024-04-01                   808.120850
          2024-07-01                    55.816742
          2024-10-01                    58.705502
          2025-01-01                    52.340435
          2025-04-01                    48.178661
          2025-07-01                    50.549999
          2025-08-28 18:09:33+03:00     49.700001
Name: adjclose

C:\Users\okand\anaconda3\Lib\site-packages\yahooquery\utils\__init__.py:1470: FutureWarning: 'S' is deprecated and will be removed in a future version. Please use 's' instead of 'S'.
  has_live_indice = index_utc[-1] >= last_trade - pd.Timedelta(2, "S")
C:\Users\okand\anaconda3\Lib\site-packages\yahooquery\ticker.py:1333: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df["dividends"].fillna(0, inplace=True)
C:\Users\okand\anaconda3\Lib\site-packages\yahooquery\ticker.py:1335: FutureWarning: A value is trying to be set on a copy